In [1]:
import socket
import numpy as np
import hmac

def verify_mac(block, received_mac, key, X, Y):
    for tag_index in range(X.shape[1]):
        if Y[block][tag_index] == 1:
            selected_blocks = X[:, tag_index] == 1
            data = b''.join([received_blocks[i] for i in range(len(received_blocks)) if selected_blocks[i]])
            calculated_mac = hmac.new(key, data, digestmod='sha384').digest()
            return received_mac == calculated_mac
    return False

def analyze_results(results):
    total_messages = len(results)
    verified_messages = sum(results)
    unverified_messages = total_messages - verified_messages

    print(f"Total messages received: {total_messages}")
    print(f"Verified messages: {verified_messages}")
    print(f"Unverified messages: {unverified_messages}")
    print(f"Verification success rate: {(verified_messages / total_messages) * 100:.2f}%")

def receive_and_verify(sock, X, Y, key):
    received_blocks = {}
    results = []
    
    while True:
        data, _ = sock.recvfrom(4096)
        if data == b'END':
            break
        
        FN = int.from_bytes(data[:4], 'big')
        message = data[4:]
        
        block_index = (FN - 1) % X.shape[0]
        mac_length = 48  # HMAC-SHA384 produces 48-byte digests
        
        if len(message) > mac_length:
            block_data = message[:-mac_length]
            received_mac = message[-mac_length:]
        else:
            block_data = message
            received_mac = None
        
        received_blocks[block_index] = block_data
        
        if received_mac and verify_mac(block_index, received_mac, key, X, Y):
            results.append(True)
        else:
            results.append(False)

    analyze_results(results)

# Matrix Definitions (X and Y) - should be the same as in the sender script
X = np.array([[1, 0, 0, 0, 0, 0, 1, 0, 0],  # m1
              [1, 0, 0, 0, 0, 0, 0, 1, 0],  # m2
              [1, 0, 0, 0, 0, 0, 0, 0, 1],  # m3
              [0, 1, 0, 0, 0, 0, 1, 0, 0],  # m4
              [0, 1, 0, 0, 0, 0, 0, 1, 0],  # m5
              [0, 1, 0, 0, 0, 0, 0, 0, 1],  # m6
              [0, 0, 1, 0, 0, 0, 1, 0, 0],  # m7
              [0, 0, 1, 0, 0, 0, 0, 1, 0],  # m8
              [0, 0, 1, 0, 0, 0, 0, 0, 1],  # m9
              [0, 0, 0, 1, 0, 0, 1, 0, 0],  # m10
              [0, 0, 0, 1, 0, 0, 0, 1, 0],  # m11
              [0, 0, 0, 1, 0, 0, 0, 0, 1],  # m12
              [0, 0, 0, 0, 1, 0, 1, 0, 0],  # m13
              [0, 0, 0, 0, 1, 0, 0, 1, 0],  # m14
              [0, 0, 0, 0, 1, 0, 0, 0, 1],  # m15
              [0, 0, 0, 0, 0, 1, 1, 0, 0],  # m16
              [0, 0, 0, 0, 0, 1, 0, 1, 0],  # m17
              [0, 0, 0, 0, 0, 1, 0, 0, 1]]) # m18

Y = np.array([[0, 0, 0, 0, 0, 0, 1, 0, 0],  # m1
              [0, 0, 0, 0, 0, 0, 0, 1, 0],  # m2
              [1, 0, 0, 0, 0, 0, 0, 0, 0],  # m3
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m4
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m5
              [0, 1, 0, 0, 0, 0, 0, 0, 0],  # m6
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m7
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m8
              [0, 0, 1, 0, 0, 0, 0, 0, 0],  # m9
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m10
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m11
              [0, 0, 0, 1, 0, 0, 0, 0, 0],  # m12
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m13
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m14
              [0, 0, 0, 0, 1, 0, 0, 0, 0],  # m15
              [0, 0, 0, 0, 0, 0, 0, 0, 0],  # m16
              [0, 0, 0, 0, 0, 1, 0, 0, 0],  # m17
              [0, 0, 0, 0, 0, 0, 0, 0, 1]]) # m18

sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind(("10.42.0.15", 23422))

key = b"key"

print("Listening for messages...")

receive_and_verify(sock, X, Y, key)


OSError: [Errno 99] Cannot assign requested address